# Slit Annular plate with TDNNS formulation
In this notebook we consider a slit annular plate, where one of the slit parts are clamped whereas on the other a vertical force $P$ is applied. The inner and outer boundaries are left free.

The material and geometrical properties are
$$
E = 2.1\times 10^7,\qquad \nu = 0,\qquad R_i=6,\qquad R_o=10, \qquad t=0.03,
$$
the maximal shear force is $P_{\max}=0.8 = \frac{3.2}{R_o-R_i}$ (force/length of edge), and we use the TDNNS method for nonlinear Naghdi shells.

In [ ]:
# Load NGSolve module, and the Open Cascade (OCC) module
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import ipywidgets as widgets

# Geometry and material parameters
thickness = 0.03
R_o = 10
R_i = 6
E, nu = 2.1e7, 0
G = E / (2 * (1 + nu))
kappa = 5 / 6
# Maximal shear force (force/length)
shear = CF((0, 0, 3.2 / (R_o - R_i)))

# Create geometry and mesh
# Angle close to 360 to avoid touching
angle = 359.999
shape = (
    WorkPlane()
    .MoveTo(0, -R_o)
    .Arc(R_o, angle)
    .Rotate(90)
    .Line(R_o - R_i)
    .Rotate(90)
    .Arc(R_i, -angle)
    .Rotate(90)
    .Line(R_o - R_i)
    .Face()
)

# Naming edges
for edge in shape.edges:
    rho, c = edge.Properties()
    if abs(c.y + 0.5 * (R_o + R_i)) < 1e-5:
        if c.x < -1e-8:
            edge.name = "force"
        else:
            edge.name = "clamped"
    elif rho < 50:
        edge.name = "inner"
    else:
        edge.name = "outer"

# maxh = 2, 0.25
mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=0.25)).Curve(2)
Draw(mesh);

At the clamped boundary we need to fix the displacement by homogeneous Dirichlet boundary conditions, the shearing field, and the hybridization variable, which has the physical meaning of the rotated angle.

In [ ]:
# Set polynomial order for displacement field u. The bending moment
# tensor m, shear gamma, and hybrid variable hyb have one degree less
order = 1
# curve mesh according to displacement order
mesh.Curve(order)

# Create finite element spaces
# Clamped at "clamped" edge (bbnd ... co-dimension 2 boundary of 3D mesh)
fesU = VectorH1(mesh, order=order, dirichlet_bbnd="clamped")
fesM = HDivDivSurface(mesh, order=order - 1, discontinuous=True)
fesH = NormalFacetSurface(mesh, order=order - 1, dirichlet_bbnd="clamped")
fesB = HCurl(mesh, order=order - 1, dirichlet_bbnd="clamped")
fes = fesU * fesM * fesB * fesH

# Symbolic trial functions for displacement field u,
# bending moment tensor m, shear gamma, and hybrid variable hyb
u, m, gamma, hyb = fes.TrialFunction()
# Trace needed for m, gamma, and hyb as we are on the surface
m, gamma, hyb = m.Trace(), gamma.Trace(), hyb.Trace()

# Regge FESpace to avoid membrane locking
Regge = HCurlCurl(mesh, order=order - 1, discontinuous=True)

# GridFunction to store the solution
gf_solution = GridFunction(fes, name="solution")

We define the unit normal vector $\hat\nu$, edge-tangential vector $\hat \tau$ and the co-normal vector $\hat\mu = \hat\nu\times \hat \tau$ at the initial configuration.

Then the projection operator onto the tangent space, deformation gradient, Cauchy-Green, and Green tensors $\boldsymbol{P}$, $\boldsymbol{F}$, $\boldsymbol{C}$, and $\boldsymbol{E}$ are introduced.

Finally, the unit normal, edge-tangential, and co-normal vectors $\nu, \tau,\mu$ on the deformed configuration are declared, which depend on the unknown displacement field $u$.

In [ ]:
# Surface unit normal, edge-tangential, and co-normal vectors on initial configuration
nv = specialcf.normal(mesh.dim)
tv = specialcf.tangential(mesh.dim)
cnv = Cross(nv, tv)

# Projection to the surface tangent space
Ptau = Id(mesh.dim) - OuterProduct(nv, nv)
# Surface deformation gradient
Ftau = Grad(u).Trace() + Ptau
# Surface Cauchy-Green tensor
Ctau = Ftau.trans * Ftau
# Surface Green-Lagrange strain tensor
Etautau = 0.5 * (Ctau - Ptau)
# surface determinant
J = Norm(Cof(Ftau) * nv)


def PseudoInverse(mat, v):
    """Pseudo Inverse of a rank (n-1) matrix
    v needs to lie in the kernel of mat
    """
    return Inv(mat.trans * mat + OuterProduct(v, v)) * mat.trans


# Surface unit normal, edge-tangential, co-normal vectors, and director on deformed configuration
nv_def = Normalize(Cof(Ftau) * nv)
tv_def = Normalize(Ftau * tv)
cnv_def = Cross(nv_def, tv_def)
director = nv_def + PseudoInverse(Ftau, nv).trans * gamma

# Surface Hessian weighted with director on deformed configuration
H_nv_def = (u.Operator("hesseboundary").trans * director).Reshape((3, 3))

For the angle computation of the bending energy we use an averaged unit normal vector avoiding the necessity of using information of two neighbored element at once (+ a more stable formulation using the co-normal vector instead of the unit normal vector)

<center><img src="pictures/nonsmooth_av_nv_el_nv.png" width="150"> </center>

\begin{align*}
\sum_{E\in\mathcal{E}_h}\int_E(\sphericalangle(\hat{\nu}_L,\hat{\nu}_R)-\sphericalangle(\nu_L,\nu_R)\circ\phi)\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,ds &= \sum_{T\in\mathcal{T}_h}\int_{\partial T}(\sphericalangle(\mu\circ\phi,P^{\perp}_{\tau\circ\phi}(\{\!\{\nu^n\}\!\}))-\sphericalangle(\hat{\mu},\{\!\{\hat{\nu}\}\!\}))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,ds,
\end{align*}
where 
$$
P^{\perp}_{\tau\circ\phi}(v)= \frac{1}{\|\boldsymbol{P}^{\perp}_{\tau\circ\phi}v\|}\boldsymbol{P}^{\perp}_{\tau\circ\phi}v,\qquad \boldsymbol{P}^{\perp}_{\tau\circ\phi}= \boldsymbol{I}-\tau\circ\phi\otimes\tau\circ\phi
$$
denotes the (nonlinear) normalized projection to the plane perpendicular to the deformed edge-tangential vector $\tau$ for measuring the correct angle.

In [ ]:
# Save clamped boundary for updating averaged unit normal vector during load-steps
gf_clamped_bnd = GridFunction(FacetSurface(mesh, order=0))
gf_clamped_bnd.Set(1, definedon=mesh.BBoundaries("left"))

# Unit normal vector on current configuration
# Used to update averaged unit normal vector during load-steps
cf_nv_cur = Normalize(Cof(Ptau + Grad(gf_solution.components[0])) * nv)

# FESpace for averaged normal vectors living only on the edges of the mesh
fesVF = VectorFacetSurface(mesh, order=order - 1)
# GridFunctions to save averaged normal vectors on deformed and initial configurations
averaged_nv = GridFunction(fesVF)
averaged_nv_init = GridFunction(fesVF)

# Initialize by averaging unit normal vectors on initial configuration
# definedon=mesh.Boundaries(".*") is needed as we interpolate on the surface mesh
averaged_nv.Set(nv, dual=True, definedon=mesh.Boundaries(".*"))
averaged_nv_init.Set(nv, dual=True, definedon=mesh.Boundaries(".*"))
# Normalize averaged normal vector on initial configuration
cf_averaged_nv_init_norm = Normalize(averaged_nv_init)
# Project averaged unit normal vector being perpendicular to deformed edge-tangent vector
# to measure correct angle on deformed configuration
cf_proj_averaged_nv = Normalize(averaged_nv - (tv_def * averaged_nv) * tv_def)

Define the material and inverse material norms $\|\cdot\|_{\mathbb{M}}^2$ and $\|\cdot\|_{\mathbb{M}^{-1}}^2$ with Young modulus $\bar{E}$ and Poisson's ratio $\bar{\nu}$
\begin{align*}
\mathbb{M} \boldsymbol{E} = \frac{\bar E}{1-\bar \nu^2}\big((1-\bar \nu)\boldsymbol{E}+\bar \nu\,\mathrm{tr}(\boldsymbol{E})\boldsymbol{P}\big),\qquad\mathbb{M}^{-1} \boldsymbol{m} = \frac{1+\bar \nu}{\bar E}\big(\boldsymbol{m}-\frac{\bar \nu}{\bar\nu+1}\,\mathrm{tr}(\boldsymbol{m})\boldsymbol{P}\big).
\end{align*}

In [ ]:
# Material norm
def MaterialNorm(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * InnerProduct(mat, mat) + nu * Trace(mat) ** 2)


# Material stress
def MaterialStress(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * mat + nu * Trace(mat) * Ptau)


# Inverse of the material norm
def MaterialNormInv(mat, E, nu):
    return (1 + nu) / E * (InnerProduct(mat, mat) - nu / (nu + 1) * Trace(mat) ** 2)

Define shell energies, where the bending energy is incorporated as a saddle point problem. We set ``condense=True`` in the bilinear form to compute the Schur complement eliminating the bending moment tensor unknowns $\boldsymbol{m}$ from the global system. Thus, we obtain a minimization problem in $(u,\alpha,\hat{\gamma})$.

In [ ]:
# Bilinear form for problem
# We define the Lagrangian of the TDNNS formulation. Therefore,
# we use Variation() such that Newton's method knows to build the first variation
# for the residual and the second variation for the stiffness matrix. The
# stiffness matrix  will be symmetric and we use static condensation via
# condense = True to eliminate the bending moment tensor m from the global system.
# We use .Compile() to simplify (linearize) the coefficient expression tree.

bfA = BilinearForm(fes, symmetric=True, condense=True)
# Membrane energy
# Interpolate the membrane strains Etautau into the Regge space
# to avoid membrane locking
bfA += Variation(
    0.5 * thickness * MaterialNorm(Interpolate(Etautau, Regge), E, nu) * ds
).Compile()
# Bending energy
# Element terms of bending energy
bfA += Variation(
    (
        -6 / thickness**3 * MaterialNormInv(m, E, nu)
        + InnerProduct(
            H_nv_def + (J - nv * director) * Grad(nv) - Grad(gamma), 1 / J * m
        )
    )
    * ds
).Compile()
# Boundary terms of bending energy including hybridization variable
bfA += Variation(
    (
        acos(cnv_def * cf_proj_averaged_nv)
        - acos(cnv * cf_averaged_nv_init_norm)
        + hyb * cnv
        + (PseudoInverse(Ftau, nv).trans * gamma) * cnv_def
    )
    * m[cnv, cnv]
    * ds(element_boundary=True)
).Compile()
# Shear energy
bfA += Variation(0.5 * thickness * kappa * G * gamma * gamma * ds)


# Shear force. Parameter par for load-stepping below.
par = Parameter(0.0)
bfA += Variation(-par * shear * u * dx(definedon=mesh.BBoundaries("force")))

In [ ]:
# Reset the solution to zero
gf_solution.vec[:] = 0
# Extract components of solution
gfu, gfm, gfgamma, _ = gf_solution.components

# Draw displacement, bending moment tensor, shear stresses, and membrane stresses
scene = Draw(gfu, mesh, "displacement", deformation=gfu)
scene2 = Draw(Norm(gfm), mesh, "bending_stress", deformation=gfu)
scene3 = Draw(thickness * kappa * G * gfgamma, mesh, "shear_stress", deformation=gfu)
gf_membrane_strain = GridFunction(Regge)
gf_membrane_strain.Set(
    0.5 * (Grad(gfu).trans * Grad(gfu) + Grad(gfu).trans * Ptau + Ptau * Grad(gfu)),
    dual=True,
    definedon=mesh.Boundaries(".*"),
)
scene4 = Draw(
    Norm(thickness * MaterialStress(gf_membrane_strain, E, nu)),
    mesh,
    "membrane_stress",
    deformation=gfu,
)

Use Newton's method for solving and increase magnitude of right-hand side by load-steps.

The unit normal vector is averaged after each load-step.

In [ ]:
# Declare the two points of interest
point_A = (
    cos(angle / 180 * pi - pi / 2) * R_o,
    sin(angle / 180 * pi - pi / 2) * R_o,
    0,
)
point_B = (
    cos(angle / 180 * pi - pi / 2) * R_i,
    sin(angle / 180 * pi - pi / 2) * R_i,
    0,
)
result_A = [(0, 0, 0)]
result_B = [(0, 0, 0)]

# Use num_steps uniform load-steps in [0,1]
num_steps = 20

tw = widgets.Text(value="step = 0")
display(tw)

# Thread parallel
with TaskManager():
    for steps in range(num_steps):
        par.Set((steps + 1) / num_steps)
        tw.value = f"step = {steps+1}/{num_steps}"

        # Update averaged normal vector
        # On clamped boundary it remains the unit normal vector of the initial configuration
        # and on the rest of the boundary it is updated by the current unit normal vector
        averaged_nv.Set(
            (1 - gf_clamped_bnd) * cf_nv_cur + gf_clamped_bnd * nv,
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )

        # Use Newton solver with residual tolerance 1e-5 and maximal 100 iterations
        # Due to hybridization techniques we can use the sparsecholesky solver for
        # solving the Schur complement (done internally)
        # We use a damping factor of 1/3, i.e. the search direction residuum is
        # scaled with min(1/3*iteration, 1)
        solvers.Newton(
            bfA,
            gf_solution,
            inverse="sparsecholesky",
            dampfactor=1 / 3,
            printing=False,
            maxerr=1e-5,
            maxit=100,
        )
        # Redraw solutions
        scene.Redraw()
        scene2.Redraw()
        scene3.Redraw()
        gf_membrane_strain.Set(
            0.5
            * (Grad(gfu).trans * Grad(gfu) + Grad(gfu).trans * Ptau + Ptau * Grad(gfu)),
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )
        scene4.Redraw()

        result_A.append((gfu(mesh(*point_A, BND))))
        result_B.append((gfu(mesh(*point_B, BND))))

Reference solution taken from Sze, Liu, Lo, "Popular Benchmark Problems for Geometric Nonlinear Analysis of Shells", Finite Elements in Analysis and Design, 40(11), 1551-1569, 2004.

In [ ]:
ref_Az = [
    (0, 0.0),
    (1.789, 0.025),
    (3.37, 0.05),
    (4.72, 0.075),
    (5.876, 0.1),
    (6.872, 0.125),
    (7.736, 0.15),
    (9.16, 0.2),
    (10.288, 0.25),
    (11.213, 0.3),
    (11.992, 0.35),
    (12.661, 0.4),
    (13.247, 0.45),
    (13.768, 0.5),
    (14.24, 0.55),
    (14.674, 0.6),
    (15.081, 0.65),
    (15.469, 0.7),
    (15.842, 0.75),
    (16.202, 0.8),
    (16.55, 0.85),
    (16.886, 0.9),
    (17.212, 0.95),
    (17.528, 1.0),
]

ref_Bz = [
    (0, 0.0),
    (1.305, 0.025),
    (2.455, 0.05),
    (3.435, 0.075),
    (4.277, 0.1),
    (5.007, 0.125),
    (5.649, 0.15),
    (6.725, 0.2),
    (7.602, 0.25),
    (8.34, 0.3),
    (8.974, 0.35),
    (9.529, 0.4),
    (10.023, 0.45),
    (10.468, 0.5),
    (10.876, 0.55),
    (11.257, 0.6),
    (11.62, 0.65),
    (11.97, 0.7),
    (12.31, 0.75),
    (12.642, 0.8),
    (12.966, 0.85),
    (13.282, 0.9),
    (13.59, 0.95),
    (13.891, 1.0),
]

Plot the solution and compare with reference values.

In [ ]:
import matplotlib.pyplot as plt

_, _, A_z = zip(*result_A)
y_axis = [i / num_steps for i in range(len(A_z))]
_, _, B_z = zip(*result_B)


A_z_ex, y_axis_ref = zip(*ref_Az)
B_z_ex, _ = zip(*ref_Bz)

plt.plot(A_z, y_axis, "-*", label="$A_z$")
plt.plot(B_z, y_axis, "-x", label="$B_z$")
plt.plot(A_z_ex, y_axis_ref, "-", label="Sze et al. 2004")
plt.plot(B_z_ex, y_axis_ref, "-", label="Sze et al. 2004")

plt.xlabel("displacement")
plt.ylabel("$P/P_{\\max}$")
plt.legend()
plt.show()

In [ ]:
print("A_z \n")
for i, (az, y) in enumerate(zip(A_z, y_axis)):
    print(f"({az}, {y} )")


print("\n\n B_z \n")
for i, (bz, y) in enumerate(zip(B_z, y_axis)):
    print(f"({bz}, {y} )")